# LCA without foreground

In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from bw_exiobase.background_importer import *
from bw_exiobase.foreground_importer import *
from bw_exiobase.datapackage_builder import *
from bw_exiobase.uncertainty_handler import *
from bw_exiobase.lca_batch import *
from constants import *

1. Get technosphere matrix

In [2]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.form_tech_matrix(raw_tech)

2. Get biosphere matrix

In [3]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.form_bio_matrix(bio_df, GHG)

3. Get characterization factor matrix

In [4]:
cf_matrix = bg_importer.form_cf_matrix(EMISSION_CODE_FILE, METHOD, GHG)

## Run simulation with only background

1. Build datapackages

In [5]:
dp_builder = DatapackageBuilder()
datapackage_data = dp_builder.prepare_bw_matrix(tech_matrix, bio_matrix, cf_matrix)
datapackage = dp_builder.prepare_datapackage(datapackage_data)

2. Get background activities

In [6]:
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
bg_activities

['EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
 'EU28-Industry',
 'EU28-Motor Gasoline',
 'EU28-Gas/Diesel Oil',
 'EU28-Refinery Feedstocks',
 'EU28-Lubricants',
 'EU28-P- and other fertiliser',
 'EU28-Chemicals nec',
 'EU28-Biogasoline',
 'EU28-Biodiesels',
 'EU28-Cement, lime and plaster',
 'EU28-Basic iron and steel and of ferro-alloys and first products thereof',
 'EU28-Aluminium and aluminium products',
 'EU28-Copper products',
 'EU28-Electricity by coal',
 'EU28-Electricity by gas',
 'EU28-Electricity by nuclear',
 'EU28-Electricity by hydro',
 'EU28-Electricity by wind',
 'EU28-Electricity by petroleum and other oil derivatives',
 'EU28-Electricity by biomass and waste',
 'EU28-Electricity by solar photovoltaic',
 'EU28-Electricity by solar thermal',
 'EU28-Electricity by tide, wave, ocean',
 'EU28-Electricity by Geothermal',
 'EU28-Electricity nec',
 'EU28-Steam and hot water supply 

3. Run static simulations

In [7]:
mysector = 0

lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[datapackage],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, bg_activities[mysector]}")


Brightway calculated lca score: (681059.4825065059, 'EU28-Agriculture-Forestry-Fishing')


## Add foreground system with 1 column

1. Check out the foreground data

In [8]:
fg_df = pd.read_csv(FG_FILE_1COL, delimiter=",")
fg_df.head()

,Activity name,Exchange name,Exchange type,Exchange amount,Exchange uncertainty type,u,Exchange negative
0,column_1,EU28-Agriculture-Forestry-Fishing,technosphere,0.000000,0,NaN,NaN
1,column_1,EU28-Energy,technosphere,NaN,0,NaN,NaN
2,column_1,EU28-Natural gas and services related to natur...,technosphere,NaN,0,NaN,NaN
3,column_1,EU28-Industry,technosphere,0.000001,2,3.330005,False
4,column_1,EU28-Motor Gasoline,technosphere,-0.000016,2,1.540000,True


2. Get activities in foreground system.

In [9]:
fg_activities = get_fg_activities(FG_FILE_1COL, ",")
fg_activities

['column_1']

3. Get activities in background system.

In [10]:
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
bg_activities

['EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
 'EU28-Industry',
 'EU28-Motor Gasoline',
 'EU28-Gas/Diesel Oil',
 'EU28-Refinery Feedstocks',
 'EU28-Lubricants',
 'EU28-P- and other fertiliser',
 'EU28-Chemicals nec',
 'EU28-Biogasoline',
 'EU28-Biodiesels',
 'EU28-Cement, lime and plaster',
 'EU28-Basic iron and steel and of ferro-alloys and first products thereof',
 'EU28-Aluminium and aluminium products',
 'EU28-Copper products',
 'EU28-Electricity by coal',
 'EU28-Electricity by gas',
 'EU28-Electricity by nuclear',
 'EU28-Electricity by hydro',
 'EU28-Electricity by wind',
 'EU28-Electricity by petroleum and other oil derivatives',
 'EU28-Electricity by biomass and waste',
 'EU28-Electricity by solar photovoltaic',
 'EU28-Electricity by solar thermal',
 'EU28-Electricity by tide, wave, ocean',
 'EU28-Electricity by Geothermal',
 'EU28-Electricity nec',
 'EU28-Steam and hot water supply 

4. Background matrices

In [11]:
print(tech_matrix)
print("---")
print(bio_matrix)

[[7.36338343e-01 1.32512793e-03 2.50369329e-03 ... 1.21134511e-04
  7.62826166e-05 4.74480856e-04]
 [9.91365844e-03 8.30899707e-01 3.02023124e-02 ... 9.94126465e-05
  5.41113056e-05 9.39990163e-05]
 [4.32464974e-05 2.21849241e-03 9.99194240e-01 ... 3.70894886e-08
  2.00023023e-08 7.04757337e-09]
 ...
 [6.34806533e-06 8.26437110e-06 7.47308387e-07 ... 9.86260390e-01
  1.12285354e-04 3.46522374e-03]
 [1.23636432e-05 1.94495460e-05 2.45543251e-06 ... 5.76682882e-04
  9.87318876e-01 2.70409581e-03]
 [6.07845068e-06 1.32530898e-05 1.91153928e-06 ... 6.24171469e-03
  9.40994608e-04 9.95178298e-01]]
---
[[6.92719535e+04 2.24902521e+05 1.58717207e+06 ... 8.11463102e+04
  5.09153559e+04 5.02862192e+04]
 [1.11657045e+02 3.26772076e+02 1.45794545e+01 ... 4.06909320e+03
  1.11515389e+03 4.31986073e+03]
 [5.76029465e+01 6.48498046e+01 6.86348286e+00 ... 1.11483525e+03
  3.28857868e+02 1.21602199e+03]
 ...
 [2.19718421e+00 3.01196685e+00 3.69253098e+00 ... 1.38856482e+00
  6.37694131e-01 7.10205514e

5. Concatenate foreground to background.

In [12]:
fg_importer = ForegroundImporter()
fgbg, fgfg, bgfg, bifg = fg_importer.extend_matrix(fg_df, GHG, fg_activities, bg_activities)
tech_matrix_withfg, bio_matrix_withfg = fg_importer.concatenate_matrix(tech_matrix, bio_matrix, fgbg, fgfg, bgfg, bifg)

6. Check out the matrices after concatenation.

In [13]:
tech_matrix_withfg[:, 0]

array([ 1.20000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        1.21500e-06, -1.55000e-05, -1.97000e-05,  0.00000e+00,
        1.53000e-10, -6.46000e-05,  2.61268e-06,  0.00000e+00,
        5.96124e-06,  0.00000e+00,  0.00000e+00,  1.49850e-05,
        0.00000e+00, -5.84000e-07, -3.91000e-07, -2.62000e-07,
       -1.03000e-07, -5.37000e-07, -1.92000e-08, -2.34000e-07,
       -2.01000e-07,  0.00000e+00, -6.78000e-09, -9.35000e-10,
        0.00000e+00,  7.55000e-07,  1.80000e-07,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  2.21100e-06,  1.65196e-05,
       -1.00000e-05,  5.67000e-07,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.000

In [14]:
print(tech_matrix_withfg)
print("---")
print(bio_matrix_withfg)

[[1.20000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.36338343e-01 1.32512793e-03 ... 1.21134511e-04
  7.62826166e-05 4.74480856e-04]
 [0.00000000e+00 9.91365844e-03 8.30899707e-01 ... 9.94126465e-05
  5.41113056e-05 9.39990163e-05]
 ...
 [0.00000000e+00 6.34806533e-06 8.26437110e-06 ... 9.86260390e-01
  1.12285354e-04 3.46522374e-03]
 [0.00000000e+00 1.23636432e-05 1.94495460e-05 ... 5.76682882e-04
  9.87318876e-01 2.70409581e-03]
 [0.00000000e+00 6.07845068e-06 1.32530898e-05 ... 6.24171469e-03
  9.40994608e-04 9.95178298e-01]]
---
[[0.00000000e+00 6.92719535e+04 2.24902521e+05 ... 8.11463102e+04
  5.09153559e+04 5.02862192e+04]
 [0.00000000e+00 1.11657045e+02 3.26772076e+02 ... 4.06909320e+03
  1.11515389e+03 4.31986073e+03]
 [0.00000000e+00 5.76029465e+01 6.48498046e+01 ... 1.11483525e+03
  3.28857868e+02 1.21602199e+03]
 ...
 [0.00000000e+00 2.19718421e+00 3.01196685e+00 ... 1.38856482e+00
  6.37694131e-01 7.10205514e

7. Build datapackage

In [15]:
dp_builder = DatapackageBuilder()
datapackage_data_withfg = dp_builder.prepare_bw_matrix(tech_matrix_withfg, bio_matrix_withfg, cf_matrix)
datapackage_withfg = dp_builder.prepare_datapackage(datapackage_data_withfg)

8. Run static simulation

In [16]:
mysector = 0

activities = fg_activities + bg_activities

lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[datapackage_withfg],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, activities[mysector]}")


Brightway calculated lca score: (-82.01023771255333, 'column_1')


## Add foreground system with 2 columns

1. Check out the foreground data

In [17]:
fg_df = pd.read_csv(FG_FILE_2COL, delimiter=",")
fg_df.head()

,Activity name,Exchange name,Exchange type,Exchange amount,Exchange uncertainty type,GSD,Exchange negative
0,column_1,EU28-Electricity by biomass and waste,technosphere,-2.340000e-07,2,2.674795,True
1,column_1,EU28-Electricity by solar photovoltaic,technosphere,-2.010000e-07,2,2.674795,True
2,column_1,EU28-Electricity by solar thermal,technosphere,0.000000e+00,1,0.000000,False
3,column_1,"EU28-Electricity by tide, wave, ocean",technosphere,-6.780000e-09,2,2.674795,True
4,column_1,EU28-Electricity by Geothermal,technosphere,-9.350000e-10,2,2.674795,True


2. Get activities in foreground system.

In [18]:
fg_activities = get_fg_activities(FG_FILE_2COL, ",")
fg_activities

['column_1', 'column_2']

3. Get activities in background system.

In [19]:
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
bg_activities

['EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
 'EU28-Industry',
 'EU28-Motor Gasoline',
 'EU28-Gas/Diesel Oil',
 'EU28-Refinery Feedstocks',
 'EU28-Lubricants',
 'EU28-P- and other fertiliser',
 'EU28-Chemicals nec',
 'EU28-Biogasoline',
 'EU28-Biodiesels',
 'EU28-Cement, lime and plaster',
 'EU28-Basic iron and steel and of ferro-alloys and first products thereof',
 'EU28-Aluminium and aluminium products',
 'EU28-Copper products',
 'EU28-Electricity by coal',
 'EU28-Electricity by gas',
 'EU28-Electricity by nuclear',
 'EU28-Electricity by hydro',
 'EU28-Electricity by wind',
 'EU28-Electricity by petroleum and other oil derivatives',
 'EU28-Electricity by biomass and waste',
 'EU28-Electricity by solar photovoltaic',
 'EU28-Electricity by solar thermal',
 'EU28-Electricity by tide, wave, ocean',
 'EU28-Electricity by Geothermal',
 'EU28-Electricity nec',
 'EU28-Steam and hot water supply 

4. Background matrices

In [20]:
print(tech_matrix)
print("---")
print(bio_matrix)

[[7.36338343e-01 1.32512793e-03 2.50369329e-03 ... 1.21134511e-04
  7.62826166e-05 4.74480856e-04]
 [9.91365844e-03 8.30899707e-01 3.02023124e-02 ... 9.94126465e-05
  5.41113056e-05 9.39990163e-05]
 [4.32464974e-05 2.21849241e-03 9.99194240e-01 ... 3.70894886e-08
  2.00023023e-08 7.04757337e-09]
 ...
 [6.34806533e-06 8.26437110e-06 7.47308387e-07 ... 9.86260390e-01
  1.12285354e-04 3.46522374e-03]
 [1.23636432e-05 1.94495460e-05 2.45543251e-06 ... 5.76682882e-04
  9.87318876e-01 2.70409581e-03]
 [6.07845068e-06 1.32530898e-05 1.91153928e-06 ... 6.24171469e-03
  9.40994608e-04 9.95178298e-01]]
---
[[6.92719535e+04 2.24902521e+05 1.58717207e+06 ... 8.11463102e+04
  5.09153559e+04 5.02862192e+04]
 [1.11657045e+02 3.26772076e+02 1.45794545e+01 ... 4.06909320e+03
  1.11515389e+03 4.31986073e+03]
 [5.76029465e+01 6.48498046e+01 6.86348286e+00 ... 1.11483525e+03
  3.28857868e+02 1.21602199e+03]
 ...
 [2.19718421e+00 3.01196685e+00 3.69253098e+00 ... 1.38856482e+00
  6.37694131e-01 7.10205514e

5. Concatenate foreground to background.

In [21]:
fg_importer = ForegroundImporter()
fgbg, fgfg, bgfg, bifg = fg_importer.extend_matrix(fg_df, GHG, fg_activities, bg_activities)
tech_matrix_withfg2, bio_matrix_withfg2 = fg_importer.concatenate_matrix(tech_matrix, bio_matrix, fgbg, fgfg, bgfg, bifg)

6. Check out the matrices after concatenation.

In [22]:
print(tech_matrix_withfg2)
print("---")
print(bio_matrix_withfg2)

[[1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 7.36338343e-01 ... 1.21134511e-04
  7.62826166e-05 4.74480856e-04]
 ...
 [0.00000000e+00 0.00000000e+00 6.34806533e-06 ... 9.86260390e-01
  1.12285354e-04 3.46522374e-03]
 [0.00000000e+00 0.00000000e+00 1.23636432e-05 ... 5.76682882e-04
  9.87318876e-01 2.70409581e-03]
 [0.00000000e+00 0.00000000e+00 6.07845068e-06 ... 6.24171469e-03
  9.40994608e-04 9.95178298e-01]]
---
[[0.00000000e+00 0.00000000e+00 6.92719535e+04 ... 8.11463102e+04
  5.09153559e+04 5.02862192e+04]
 [0.00000000e+00 0.00000000e+00 1.11657045e+02 ... 4.06909320e+03
  1.11515389e+03 4.31986073e+03]
 [0.00000000e+00 0.00000000e+00 5.76029465e+01 ... 1.11483525e+03
  3.28857868e+02 1.21602199e+03]
 ...
 [5.62000000e+00 1.70000000e+00 2.19718421e+00 ... 1.38856482e+00
  6.37694131e-01 7.10205514e

7. Build datapackage

In [23]:
dp_builder = DatapackageBuilder()
datapackage_data_withfg2 = dp_builder.prepare_bw_matrix(tech_matrix_withfg2, bio_matrix_withfg2, cf_matrix)
datapackage_withfg2 = dp_builder.prepare_datapackage(datapackage_data_withfg2)

8. Run static simulation

In [24]:
mysector = 0

activities = fg_activities + bg_activities

lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[datapackage_withfg2],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, activities[mysector]}")

Brightway calculated lca score: (-30322.354618841822, 'column_1')


## Run the simulation manually to compare

1. With only background system

In [25]:
from bw_exiobase.lca_batch import *

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()

identify_matrix = np.identity(len(raw_tech))
tech_matrix_manual = identify_matrix - raw_tech

rca = ExiobaseLCA()
rca.manual_lca(tech_matrix_manual, bio_matrix, cf_matrix, 0)

681059.4825065061

2. With both foreground system (1 col) and background system

In [26]:
from bw_exiobase.lca_batch import *

np.fill_diagonal(tech_matrix_withfg, -tech_matrix_withfg.diagonal())
tech_matrix_withfg = -tech_matrix_withfg

rca = ExiobaseLCA()
rca.manual_lca(tech_matrix_withfg, bio_matrix_withfg, cf_matrix, 0)

-82.01023771255339

3. With both foreground system (2 cols) and background system

In [27]:
from bw_exiobase.lca_batch import *

np.fill_diagonal(tech_matrix_withfg2, -tech_matrix_withfg2.diagonal())
tech_matrix_withfg2 = -tech_matrix_withfg2

rca = ExiobaseLCA()
rca.manual_lca(tech_matrix_withfg2, bio_matrix_withfg2, cf_matrix, 0)

-30322.354618841786